## エンドポイントの作成と削除
### 参考
- [ステップ 6: モデルを Amazon SageMaker にデプロイする@Amazon SageMaker](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/ex1-model-deployment.html)

In [1]:
%%time
import time
import boto3
import sagemaker as sage
from time import gmtime, strftime
import numpy as np
role = sage.get_execution_role()
sm = boto3.Session().client('sagemaker')


CPU times: user 760 ms, sys: 68.6 ms, total: 829 ms
Wall time: 881 ms


In [1]:
# 学習コンテナから、デプロイ可能なモデルを作成

image = f'{accountid}.dkr.ecr.us-west-2.amazonaws.com/sagemaker-{container_name}latest'
training_job_name = '{training_job_name}'
model_name = training_job_name + '-mod'

info = sm.describe_training_job(TrainingJobName=training_job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': image,
    'ModelDataUrl': model_data
}

create_model_response = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

In [2]:
# SageMaker エンドポイント設定
endpoint_config_name = f'{model_name}' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.t2.medium',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [5]:
# エンドポイントの作成
endpoint_name = '{container_name}-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

In [6]:
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import pyarrow.parquet as pq
import s3fs

In [6]:
def s3_load(s3_path):
    # s3からデータを読み込む
    fs = s3fs.S3FileSystem()
    dataset = pq.ParquetDataset(s3_path, filesystem=fs)
    table = dataset.read()
    df = table.to_pandas()
    return df

def s3_load_days(s3_dir_path, start_dt, end_dt):
    # s3から日毎にデータを読み込む
    dfs = []
    dt = start_dt
    while dt <= end_dt:
        dt_str = dt.strftime('%Y-%m-%d')
        print(dt_str)
        s3_dt_path = s3_dir_path + f'dt={dt_str}'
        df = s3_load(s3_dt_path)
        dfs.append(df)
        dt += relativedelta(days=1)
    dfs = pd.concat(dfs)
    return dfs

In [7]:
s3_dir_path = f's3://{bucket}/{prifix}'
start_dt = datetime.datetime(2020, 1, 1)
end_dt = datetime.datetime(2020, 1, 1)
df_test = s3_load_days(s3_dir_path, start_dt, end_dt)

In [8]:
X_test = df_test[['feature']]
y_test = df_test['objective']

In [9]:
# 予測
y_pred_top3 = []
split_array = np.array_split(X_test.values, int(X_test.values.shape[0] / 50 + 1))

for i, array in enumerate(split_array):
    if i%100 == 0:
        print(f'{i}/{len(split_array)}')
    body = '\n'.join([','.join([str(item) for item in (items)]) for items in array])
    runtime_client = boto3.client('runtime.sagemaker')
    response = runtime_client.invoke_endpoint(EndpointName = endpoint_name,
                                     ContentType = 'text/csv',
                                     Body = body)
    y_pred = [pred.split(',') for pred in response['Body'].read().decode('utf8').split('\n')[:-1]]
    y_pred_top3 += y_pred
print(len(y_pred_top3))

In [17]:
# Top1件　精度
acc = 0
for test, top3 in zip(y_test, y_pred_top3):
    #print(test in top3)
    if test in top3[0:1]:
        acc += 1
    
acc/len(y_pred_top3)

0.0718031528794807

In [18]:
# Top3件　精度
acc = 0
for test, top3 in zip(y_test, y_pred_top3):
    #print(test in top3)
    if test in top3:
        acc += 1
    
acc/len(y_pred_top3)

0.14835481085920763

In [ ]:
# エンドポイント削除
client = boto3.client('sagemaker')
response = client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = response['ProductionVariants'][0]['ModelName']

client.delete_model(ModelName=model_name)    
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_config_name);